Dask and Scikit-Image on a Cluster
==================================

<img src="http://scikit-image.org/_static/img/logo.png">

In [ ]:
from dask_kubernetes import KubeCluster
cluster = KubeCluster()
cluster

In [ ]:
from dask.distributed import Client, progress
client = Client(cluster)

In [ ]:
%matplotlib inline
from dask import delayed
import skimage.io
import matplotlib.pyplot as plt

In [ ]:
sample = skimage.io.imread('https://storage.googleapis.com/pangeo-data/FIB-25/iso.00000.png')

fig = plt.figure(figsize=(10, 10))
plt.imshow(sample, cmap='gray')

In [ ]:
from dask import delayed
import dask.array as da

# filenames = ['https://storage.googleapis.com/pangeo-data/FIB-25/iso.%05d.png' % i for i in range(0, 8090)]
filenames = ['https://storage.googleapis.com/pangeo-data/FIB-25/iso.%05d.png' % i for i in range(0, 1000)]
images = [delayed(skimage.io.imread)(fn) for fn in filenames]
arrays = [da.from_delayed(im, shape=sample.shape, dtype=sample.dtype) for im in images]
x = da.stack(arrays, axis=0)
x

In [ ]:
y = x.rechunk((200, 200, 200))
y = y.persist()
progress(y)

In [ ]:
import xarray as xr
import holoviews as hv
import numpy as np
from holoviews.operation.datashader import regrid
hv.extension('bokeh')

In [ ]:
opts = {'plot': dict(xaxis=None, yaxis=None),
        'style': dict(cmap='Greys')}
hv.opts({'Image': opts})

xs = np.arange(y.shape[2])
ys = np.arange(y.shape[1])
zs = np.arange(y.shape[0])
xrda = xr.DataArray(y, coords={'x': xs, 'y': ys, 'z': zs}, dims=['z', 'y', 'x'])
hvds = hv.Dataset(xrda, vdims=hv.Dimension('ElectronDensity', range=(0, 255)))

img_xy = hvds.to(hv.Image, ['x', 'y'], dynamic=True)
img_xz = hvds.to(hv.Image, ['x', 'z'], dynamic=True)

(regrid(img_xy) + regrid(img_xz))